In [1]:
import os 
import json 

In [2]:
path='train/'

train_files = os.listdir(path)
train = json.load(open(os.path.join(path, train_files[0])))

# Extract utilities

In [24]:
def get_utility(bid, pref):
    util = 0
    issuelist = list(pref.keys())
    for issueindex, issue in enumerate(issuelist):
        # values of issue in given bid
        value = pref[issue][bid[issueindex]]
        # weight of issue in given bid
        weight = pref[issue]['weight']
        util += weight*value
    return util

# Extraction 
all_issues = train['issues']
pref1 = train['Utility1']
pref2 = train['Utility2']
all_bids = train['bids']

# Contains (round, utility_agent1, utility_agent2)
mapped_utils = []

# Parse utility values of bids
for bid in all_bids:
    r = bid['round']
    if 'accept' in bid: break
    bid_agent1 = bid['agent1'].split(',')
    bid_agent2 = bid['agent2'].split(',')
    u1 = get_utility(bid_agent1, pref1)
    u2 = get_utility(bid_agent2, pref2)
    
    mapped_utils.append((u1, u2, int(r)))

# Discritize bids

In [30]:
# Utility change to discrete move type
def delta_mapping(delta_util):
    if (abs(delta_util[0]) <= 0.05 and abs(delta_util[1]) <= 0.05):
        return 'silent'
    if (abs(delta_util[0]) <= 0.05 and delta_util[1]) > 0.05:
        return 'nice'
    if (delta_util[0] <= 0 and delta_util[1] < 0):
        return 'unfortunate'
    if (delta_util[0] > 0 and delta_util[1] < 0):
        return 'selfish'
    if (delta_util[0] > 0 and delta_util[1] > 0):
        return 'fortunate'
    if (delta_util[0] < 0 and delta_util[1] > 0):
        return 'concession'

# List of bids to list of discrete moves
def discritized_mapping(agent_bids):
    mapped_utils_discrete = []
    prev_utils = agent_bids[0]
    for new_utils in agent_bids[1:]:
        delta_util1 = new_utils[0] - prev_utils[0]
        delta_util2 = new_utils[1] - prev_utils[1]
        delta_util = (delta_util1, delta_util2)
        mapped_utils_discrete.append(delta_mapping(delta_util))
        prev_utils = new_utils
    return mapped_utils_discrete
        
    
# Discritize bidspace
agent1_bids = [mapped_utils[i] for i in range(0, len(mapped_utils), 2)]
agent2_bids = [mapped_utils[i] for i in range(1, len(mapped_utils), 2)]

agent1_bids_discrete = discritized_mapping(agent1_bids)
agent2_bids_discrete = discritized_mapping(agent2_bids)

['unfortunate',
 'nice',
 'nice',
 'unfortunate',
 'unfortunate',
 'nice',
 'nice',
 'unfortunate',
 'unfortunate',
 'nice',
 'unfortunate',
 'silent',
 'nice',
 'nice',
 'silent',
 'unfortunate',
 'nice',
 'unfortunate',
 'nice',
 'unfortunate',
 'unfortunate',
 'nice',
 'nice',
 'unfortunate',
 'nice',
 'nice',
 'unfortunate',
 'unfortunate',
 'unfortunate',
 'nice',
 'unfortunate']

In [31]:
print(len(agent1_bids)
print(len(agent2_bids)
print()

32